# Association Rules

For our marketing and promotions team, we want to know which subset of products are the most frequent purchases of our customers. 

Given a .csv (or .xlsx) file containing all orders from the company, create a pandas dataframe wherein the rows are the Order numbers (OrderNum) and the columns are all the products. The data inside these cells contain a 1 or 0 wherein 1 would mean that the Order Number purchased the given product and 0 if otherwise.

Using this pandas dataframe, define three functions (1) Support, (2) Confidence, and (3) Lift.



In [1]:
# Initializing everything
import pandas as pd
import itertools as it
import numpy as np
import time

df = pd.read_excel(r"sample order basket.xlsx")

In [2]:
#Gets all products being sold 
product_list = list(df["Product"].unique())
product_list.sort()

print(product_list)

['apple', 'banana', 'carrot', 'eggplant']


In [3]:
# Creating the table of products for each transaction
unique_IDs = df.OrderNum.unique()
binary_table = pd.DataFrame(index = unique_IDs,columns = product_list,data=0)

for product in product_list:
    dummy_df = df[df["Product"]==product]

    for number in dummy_df["OrderNum"]:
        binary_table.loc[number,product] = 1


In [4]:
binary_table

,apple,banana,carrot,eggplant
1000,0,0,1,1
1001,1,1,0,0
1002,0,0,0,1
1003,0,0,0,1
1004,0,1,0,0
1005,0,0,0,1
1006,1,0,0,1
1007,1,1,1,0
1008,1,0,0,1
1009,0,0,0,1


In [5]:
# Defining the "Support" function
# Take note that the parameter takes in an iterable

def Support(List):
    total_rows = len(binary_table)
    
    dummy_df = binary_table.copy()
    for element in List:
        dummy_df = dummy_df[dummy_df[element.lower()]==1]
    
    instances = len(dummy_df)
    return instances/total_rows

Support(["apple","banana"])

0.2727272727272727

In [6]:
# Defining the "Confidence" function
# Take note that the parameters are strings

def Confidence(first,second):
    return Support([first.lower(),second.lower()])/Support([first.lower()])
    
Confidence("apple","banana")

0.6

In [7]:
# Defining the "Lift" function

def Lift(first,second):
    return Confidence(first,second)/Support([second.lower()])

Lift("apple","banana")

1.65

#### Original code I wrote which I thought might be needed, but later found out that it won't be necessary to code the Support, Confidence, and Lift functions.

In [8]:
# NOT NEEDED ANYMORE!
#combinations_dict is the name of all combinations given the list of products (product_list)
combinations_dict = {}

for i in range(1,len(product_list)+1):
    combinations = list(it.combinations(product_list,i))

    for grouping in combinations:
        combinations_dict[grouping] = 0

combinations_dict

{('apple',): 0,
 ('banana',): 0,
 ('carrot',): 0,
 ('eggplant',): 0,
 ('apple', 'banana'): 0,
 ('apple', 'carrot'): 0,
 ('apple', 'eggplant'): 0,
 ('banana', 'carrot'): 0,
 ('banana', 'eggplant'): 0,
 ('carrot', 'eggplant'): 0,
 ('apple', 'banana', 'carrot'): 0,
 ('apple', 'banana', 'eggplant'): 0,
 ('apple', 'carrot', 'eggplant'): 0,
 ('banana', 'carrot', 'eggplant'): 0,
 ('apple', 'banana', 'carrot', 'eggplant'): 0}

In [9]:
# NOT NEEDED ANYMORE!
# Getting the whole transaction (using a reference I found online)

def sorter(samplelist):
    samplelist.sort()
    return samplelist

def convert(samplelist):
    return tuple(i for i in samplelist)


# code from https://stackoverflow.com/questions/22219004/how-to-group-dataframe-rows-into-list-in-pandas-groupby
def f(df):
    keys, values = df.sort_values('OrderNum').values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    df2 = pd.DataFrame({'OrderNum':ukeys, 'Product':[list(a) for a in arrays]})
    df2["Product"] = df2["Product"].apply(sorter)
    df2["Product"] = df2["Product"].apply(convert)
    return df2

df2 = f(df)
df2

,OrderNum,Product
0,1000,"(carrot, eggplant)"
1,1001,"(apple, banana)"
2,1002,"(eggplant,)"
3,1003,"(eggplant,)"
4,1004,"(banana,)"
5,1005,"(eggplant,)"
6,1006,"(apple, eggplant)"
7,1007,"(apple, banana, carrot)"
8,1008,"(apple, eggplant)"
9,1009,"(eggplant,)"


In [10]:
# NOT NEEDED ANYMORE!!!
# Getting the whole transaction (second method, without using the link online)

unique_IDs = df.OrderNum.unique()

transactions = {}

for element in unique_IDs:
    items = list(df[df["OrderNum"]==element]["Product"].values)
    items.sort()
    items = tuple(items)
    transactions[str(element)] = items

df3 = pd.DataFrame(transactions.keys())
df3["Orders"] = transactions.values()
df3.columns = ["OrderNum","Products"]
df3.reset_index(inplace=True,drop=True)
df3


,OrderNum,Products
0,1000,"(carrot, eggplant)"
1,1001,"(apple, banana)"
2,1002,"(eggplant,)"
3,1003,"(eggplant,)"
4,1004,"(banana,)"
5,1005,"(eggplant,)"
6,1006,"(apple, eggplant)"
7,1007,"(apple, banana, carrot)"
8,1008,"(apple, eggplant)"
9,1009,"(eggplant,)"
